In [2]:
from web3 import Web3
import pandas as pd
pd.set_option('display.max_rows', 100)
from tqdm import tqdm
import ast #eval() to interpret code
import json
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
tqdm.pandas()
import time
import warnings
warnings.filterwarnings('ignore')
from pandas import Timestamp

In [68]:
operations = pd.read_csv('operation_all_assets_new.csv')
vau = pd.read_csv('tab_vaults.csv')

In [69]:
fold = vau[vau['fn_name']=='fold']

In [70]:
mask = (fold['ilk'] == 'ETH-A') | (fold['ilk'] == 'GUNIV3DAIUSDC2-A') | (fold['ilk'] == 'WBTC-A') | (fold['ilk'] == 'ETH-C') | (fold['ilk'] == 'ETH-B')
fold = fold[mask]
fold = fold.reset_index(drop=True)

In [71]:
fold_rate = 10 ** 27

def get_dai_rate(row):
    
    global fold_rate
    state = row['state']
    rate = int(state.split(': ')[1].split('}')[0])
    fold_rate += rate
    row['rate'] = fold_rate
    return row

fold_dai_rate = fold.progress_apply(get_dai_rate, axis=1)
fold_dai_rate

  0%|          | 0/202166 [00:00<?, ?it/s]

100%|██████████| 202166/202166 [02:36<00:00, 1289.54it/s]


,Unnamed: 0,timestamp,transaction_hash,fn_name,ilk,usr,src,state,dart,dink,rad,rate
0,0,2019-11-13 20:06:02 UTC,0xc121074d8764d1ab54187b120db3a218651da54ca0d1...,fold,ETH-A,NaN,NaN,{'rate': 0},NaN,NaN,NaN,1000000000000000000000000000
1,3,2019-11-13 21:22:33 UTC,0x6fdf3780edf0f25cd63658089ee8b99ede6de002d45a...,fold,ETH-A,NaN,NaN,{'rate': 5709754190193566796042},NaN,NaN,NaN,1000005709754190193566796042
2,7,2019-11-13 21:45:03 UTC,0xacb58d76bf55b3fb1bb46baf6bd784c95cd22fd3d39d...,fold,ETH-A,NaN,NaN,{'rate': 1678979880963078030542},NaN,NaN,NaN,1000007388734071156644826584
3,11,2019-11-13 22:07:52 UTC,0x03fa1df874381210015181649dbf10f868905b2ad7e9...,fold,ETH-A,NaN,NaN,{'rate': 1702612846924510909427},NaN,NaN,NaN,1000009091346918081155736011
4,15,2019-11-13 22:09:02 UTC,0x240f70803d5c88d90c50eba8329e17041a950e3ecb3f...,fold,ETH-A,NaN,NaN,{'rate': 87058441148993523259},NaN,NaN,NaN,1000009178405359230149259270
...,...,...,...,...,...,...,...,...,...,...,...,...
202161,2635842,2023-07-31 13:30:59 UTC,0xd2a8b781113085acb3d733ebcb3868feb906fa388355...,fold,ETH-C,NaN,NaN,{'rate': 18081010463354198051666},NaN,NaN,NaN,1366091297741194609125973542
202162,2635846,2023-07-31 13:35:59 UTC,0xe11337811b0088e052fe21ca2835ae2424423fc5a8a2...,fold,ETH-C,NaN,NaN,{'rate': 305838511913715716859},NaN,NaN,NaN,1366091603579706522841690401
202163,2635882,2023-07-31 15:18:47 UTC,0x6e072f7218a50777b4f5250919d972530eece6d41019...,fold,ETH-A,NaN,NaN,{'rate': 63261402962433981584451},NaN,NaN,NaN,1366154864982668956823274852
202164,2635944,2023-07-31 17:31:35 UTC,0x33341e5603c27ec28a4e6bb7c648bf241f9910e76fd3...,fold,ETH-C,NaN,NaN,{'rate': 14411214258490967703146},NaN,NaN,NaN,1366169276196927447790977998


In [72]:
mask = (operations['ilk'] == 'ETH-A') | (operations['ilk'] == 'GUNIV3DAIUSDC2-A') | (operations['ilk'] == 'WBTC-A') | (operations['ilk'] == 'ETH-C') | (operations['ilk'] == 'ETH-B')
operations = operations[mask]
operations = operations.reset_index(drop=True)
operations

,Unnamed: 0,usr,ilk,transaction_hash,timestamp,user_action,lock_collateral,dai,dai(in exchange)
0,0,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0a90315a4e3da0ca3e55bc9f551fbaf7ce900cb5437c...,2021-10-29 09:12:00 UTC,lock asset,90.500000,0.000000,0.000000
1,1,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0a90315a4e3da0ca3e55bc9f551fbaf7ce900cb5437c...,2021-10-29 09:12:00 UTC,generate dai,0.000000,150000.000000,141962.260382
2,2,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0xae5ebf075eca8a90246f271932bf391b8ed027b1628c...,2021-10-29 09:29:08 UTC,lock asset,34.400000,0.000000,0.000000
3,3,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0xae5ebf075eca8a90246f271932bf391b8ed027b1628c...,2021-10-29 09:29:08 UTC,generate dai,0.000000,75000.000000,70981.084371
4,4,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0c5a276fa23ca9c4b708b5630fd7d23bcb0b9c1511aa...,2021-11-21 15:46:07 UTC,payback dai,0.000000,-225317.178235,-212943.344753
...,...,...,...,...,...,...,...,...,...
578363,878021,0xffcA6A7Bb1e97E163209EaAA4Bba941b5D375b79,ETH-A,0x7e1486df5747093e22022ca8b17243b026c21b78649d...,2021-03-01 14:11:21 UTC,unlock asset,-100.000000,0.000000,0.000000
578364,878022,0xfff3d21322AA95091aAdD6064B5D591cDc2B990D,ETH-A,0x6dc580b5554d9dd18dbad5f73c620b99cd6e41990288...,2019-11-19 23:30:47 UTC,lock asset,3.139039,0.000000,0.000000
578365,878023,0xfff3d21322AA95091aAdD6064B5D591cDc2B990D,ETH-A,0x6dc580b5554d9dd18dbad5f73c620b99cd6e41990288...,2019-11-19 23:30:47 UTC,generate dai,0.000000,165.470733,165.361658
578366,878024,0xfff3d21322AA95091aAdD6064B5D591cDc2B990D,ETH-A,0x6ec359778aae7fe64ee8220b6baf8b92bb557f6ec2aa...,2020-03-25 10:36:48 UTC,payback dai,0.000000,-168.607641,-165.361658


In [73]:
def map_dai_rate(rate_tab, df_tab):
    '''
        Map dai/usd rate from fold function with table
    '''
    rate_tab['timestamp'] = pd.to_datetime(rate_tab['timestamp'])
    df_tab['timestamp'] = pd.to_datetime(df_tab['timestamp'])

    def find_closest_rate(row):
        closest_index = np.abs(rate_tab['timestamp'] - row['timestamp']).idxmin()
        rate = rate_tab['rate'][closest_index]
        return rate

    df_tab['rate'] = df_tab.progress_apply(find_closest_rate, axis=1)
    # df_tab['rate'] = df_tab['rate'].apply(lambda x: x/1e27)

    return df_tab

In [74]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        dai_rate = map_dai_rate(fold_dai_rate[fold_dai_rate.ilk == prog], operations[operations.ilk == prog])
    else:
        df_new = map_dai_rate(fold_dai_rate[fold_dai_rate.ilk == prog], operations[operations.ilk == prog])
        dai_rate = pd.concat([dai_rate, df_new], ignore_index=True, sort=False)

100%|██████████| 38683/38683 [00:23<00:00, 1637.86it/s]


In [78]:
dai_rate['rate'] = dai_rate['rate'].apply(lambda x: x/1e27)

In [79]:
dai_rate.rename(columns={"rate": "rate(DAI/USD)"}, inplace=True)

In [81]:
dai_rate.to_csv('dai_rate')

In [82]:
def calculate_dai(df):
    '''
        Calculate 'dai' value by dart(dai in exchange) * rate(dai/usd)
        Note: 
            MakerDAO doesn't have - 'dai' values in transfer in and transfer out(fork) functions
                                  - 'dai' values in liquidate(grab) function
    '''
    df['dai'] = df.apply(lambda row: row['dai(in exchange)'] * row['rate(DAI/USD)'] if row['user_action'] in ['transfer in', 'transfer out'] else row['dai'], axis=1)
    df['dai'] = df.apply(lambda row: row['dai(in exchange)'] * row['rate(DAI/USD)'] if row['user_action'] in ['liquidate'] else row['dai'], axis=1)
    return df

In [83]:
cal_dai = calculate_dai(dai_rate)
cal_dai

,Unnamed: 0,usr,ilk,transaction_hash,timestamp,user_action,lock_collateral,dai,dai(in exchange),rate(DAI/USD)
0,0,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0a90315a4e3da0ca3e55bc9f551fbaf7ce900cb5437c...,2021-10-29 09:12:00+00:00,lock asset,90.500000,0.000000,0.000000,1.182440
1,1,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0a90315a4e3da0ca3e55bc9f551fbaf7ce900cb5437c...,2021-10-29 09:12:00+00:00,generate dai,0.000000,150000.000000,141962.260382,1.182440
2,2,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0xae5ebf075eca8a90246f271932bf391b8ed027b1628c...,2021-10-29 09:29:08+00:00,lock asset,34.400000,0.000000,0.000000,1.182441
3,3,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0xae5ebf075eca8a90246f271932bf391b8ed027b1628c...,2021-10-29 09:29:08+00:00,generate dai,0.000000,75000.000000,70981.084371,1.182441
4,4,0x0000485d124CA18832ebC0E0e3D1947EE4dB8427,ETH-A,0x0c5a276fa23ca9c4b708b5630fd7d23bcb0b9c1511aa...,2021-11-21 15:46:07+00:00,payback dai,0.000000,-225317.178235,-212943.344753,1.189377
...,...,...,...,...,...,...,...,...,...,...
578363,877677,0xff7c5c4905d1768cED3bdbCfFADc435A9D766e54,ETH-B,0x3e8d7478db4624f3669185ba9242fce2cf038c6fbf23...,2022-12-16 18:19:35+00:00,unlock asset,-190.183528,0.000000,0.000000,1.301579
578364,877678,0xff7c5c4905d1768cED3bdbCfFADc435A9D766e54,ETH-B,0x93906e6bbed7108caa297009cb85d52279b5eb6a4ca9...,2022-12-17 23:30:11+00:00,lock asset,165.105319,0.000000,0.000000,1.301831
578365,877679,0xff7c5c4905d1768cED3bdbCfFADc435A9D766e54,ETH-B,0x93906e6bbed7108caa297009cb85d52279b5eb6a4ca9...,2022-12-17 23:30:11+00:00,generate dai,0.000000,138501.214316,123385.632997,1.301831
578366,877680,0xff7c5c4905d1768cED3bdbCfFADc435A9D766e54,ETH-B,0x2524ebccc2c478d9a1852672d4a541f519c7df4fdb1f...,2022-12-19 21:29:35+00:00,payback dai,0.000000,-138521.573300,-123385.632997,1.302268


In [84]:
new_col = None
cal_dai.insert(loc = 7, column = 'fee', value = new_col)

In [85]:
def cal_fee(df):

  df = df.reset_index(drop=True)
  all_usr = list(df['usr'].unique())

  for usr in tqdm(all_usr):
    tab = df[df['usr']==usr]
    sum_dai = 0
    sum_payback = 0

    for idx, row in tab.iterrows():

      if row['user_action'] == 'generate dai' or row['user_action'] == 'transfer in':
        sum_dai += row['dai']
        df['fee'].loc[idx] = 0

      elif row['user_action'] == 'payback dai' or row['user_action'] == 'transfer out':
        sum_payback += row['dai']
        df['fee'].loc[idx] = 0

      elif row['user_action'] == 'liquidate':
        liquidate = np.abs(row['dai'])

        if liquidate > (sum_dai + sum_payback):
          fee = liquidate - (sum_dai + sum_payback)
          df['fee'].loc[idx] = -fee
          df['dai'].loc[idx] = -(sum_dai + sum_payback)
          sum_dai = 0
          sum_payback = 0

        else:
          df['fee'].loc[idx] = 0

      else:
        df['fee'].loc[idx] = 0
        
  return df

In [86]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        fee = cal_fee(cal_dai[cal_dai['ilk'] == prog])
    else:
        df_new = cal_fee(cal_dai[cal_dai['ilk'] == prog])
        fee = pd.concat([fee, df_new], ignore_index=True, sort=False)

100%|██████████| 1094/1094 [00:23<00:00, 47.31it/s]


In [87]:
fee.to_csv('fee_vaults.csv')

In [89]:
rate_eth = pd.read_csv('ETH_as_of_Jul2023.csv')
rate_eth

,block_timestamp,output
0,2020-09-25 10:01:51 UTC,0x00000000000000000000000000000000000000000000...
1,2020-09-25 01:01:30 UTC,0x00000000000000000000000000000000000000000000...
2,2020-09-25 04:01:18 UTC,0x00000000000000000000000000000000000000000000...
3,2020-09-25 17:02:02 UTC,0x00000000000000000000000000000000000000000000...
4,2020-09-25 06:01:53 UTC,0x00000000000000000000000000000000000000000000...
...,...,...
91396,2023-02-19 03:01:35 UTC,0x00000000000000000000000000000000000000000000...
91397,2023-02-19 22:01:35 UTC,0x00000000000000000000000000000000000000000000...
91398,2023-02-19 20:01:23 UTC,0x00000000000000000000000000000000000000000000...
91399,2021-07-31 07:02:53 UTC,0x00000000000000000000000000000000000000000000...


In [90]:
rate_eth.rename(columns = {'block_timestamp':'timestamp'}, inplace = True)
rate_eth['timestamp'] = pd.to_datetime(rate_eth['timestamp'])
rate_eth = rate_eth.sort_values(by='timestamp', ascending=True)
rate_eth = rate_eth.reset_index(drop=True)
rate_eth['rate'] = rate_eth['output'].apply(lambda x: int(x[:66], 0)/1e18 if isinstance(x, str) and len(x) >= 66 else None)
rate_eth

,timestamp,output,rate
0,2019-11-13 20:11:19+00:00,0x00000000000000000000000000000000000000000000...,0.000000
1,2019-11-15 06:06:45+00:00,0x00000000000000000000000000000000000000000000...,181.439693
2,2019-11-15 07:05:47+00:00,0x00000000000000000000000000000000000000000000...,181.439693
3,2019-11-15 12:16:54+00:00,0x00000000000000000000000000000000000000000000...,181.439693
4,2019-11-15 13:43:45+00:00,0x00000000000000000000000000000000000000000000...,183.300000
...,...,...,...
91396,2023-07-31 23:00:23+00:00,0x00000000000000000000000000000000000000000000...,1854.800000
91397,2023-07-31 23:00:23+00:00,0x00000000000000000000000000000000000000000000...,1854.800000
91398,2023-07-31 23:00:23+00:00,0x00000000000000000000000000000000000000000000...,1854.800000
91399,2023-07-31 23:00:23+00:00,0x00000000000000000000000000000000000000000000...,1854.800000


In [91]:
rate = rate_eth.drop(columns=['output'])
rate.to_csv('USD_ETH.csv')

In [92]:
def map_eth_price_new(rate_tab, df_tab):
  '''
      To map ETH/DAI rate from osm price of MakerDAO with table 
  '''

  def find_closest_rate(row):
    closest_index = np.abs(rate_tab['timestamp'] - row['timestamp']).idxmin()
    time_rate = rate_tab['timestamp'][closest_index]
    time_data = row['timestamp']

    if time_data == time_rate:
      rate = rate_tab['rate'][closest_index]

    elif time_data < time_rate:

      if closest_index == 0:
        rate = rate_tab['rate'][0]
      else:
        rate = rate_tab['rate'][closest_index-1]

    else:

      if closest_index == rate_tab.shape[0]-1:
        rate = rate_tab['rate'][rate_tab.shape[0]-1]
      else:
        rate = rate_tab['rate'][closest_index]

    return rate

  df_tab['ETH_price'] = df_tab.progress_apply(find_closest_rate, axis=1)
  return df_tab

In [93]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        map_eth_ = map_eth_price_new(rate_eth, fee[fee['ilk'] == prog])
    else:
        df_new = map_eth_price_new(rate_eth, fee[fee['ilk'] == prog])
        map_eth_ = pd.concat([map_eth_, df_new], ignore_index=True, sort=False)

100%|██████████| 38683/38683 [01:15<00:00, 512.19it/s]


In [94]:
map_eth_.to_csv('eth_price.csv')

In [114]:
# fix case payback
def find_cr_rate_(df):
  ''' 
      To calculate collateral ratio for every transactions in debt

      Collateral Ratio by MakerDAO concept
      CR Rate = (assets[ETH] * rate[USD/ETH]) / issed dai[dai]
  '''

  df['lock_collateral'] = df['lock_collateral'].apply(lambda x: np.round(x,4))
  df['dai'] = df['dai'].apply(lambda x: np.round(x,4))
  df['fee'] = df['fee'].apply(lambda x: np.round(x,4))
  df['dai(in exchange)'] = df['dai(in exchange)'].apply(lambda x: np.round(x,4)) #to round values

  all_usr = list(df['usr'].unique())
  df['CR_Rate'] = None
  
  for usr in tqdm(all_usr):
    tab = df[df['usr']==usr]
    coll = 0
    dai = 0

    for idx, row in tab.iterrows():
      coll += row['lock_collateral']
      dai += row['dai']
      price = row['ETH_price']
      action = row['user_action']

      if action == 'liquidate': #some row['dai'] of liquidate has 0

        if row['dai'] == 0:
          cr_rate = 0
          df['CR_Rate'].loc[idx] = cr_rate
        else:
          cr_rate = np.abs(np.divide((row['lock_collateral'] * price), np.abs(row['dai'])))
          df['CR_Rate'].loc[idx] = np.round(cr_rate,4)
        coll = 0
        dai = 0

      elif action == 'payback':#check if user payback cover debt, no dai values = 0 in payback
        payback = -row['dai']
        debt = dai + payback

        if payback > debt:
          dai = 0
          cr_rate = 0
          df['CR_Rate'].loc[idx] = cr_rate
        else:
          cr_rate = np.divide((coll * price), dai)
          df['CR_Rate'].loc[idx] = np.round(cr_rate,4)

      else:

        if dai > 0:
          cr_rate = np.divide((coll * price), dai)
          df['CR_Rate'].loc[idx] = np.round(cr_rate,4)
        else:
          cr_rate = 0
          df['CR_Rate'].loc[idx] = cr_rate

  return df

In [115]:
programs = ['ETH-A', 'GUNIV3DAIUSDC2-A', 'WBTC-A', 'ETH-C', 'ETH-B']
for prog in programs:
    if prog == 'ETH-A':
        cr_rate = find_cr_rate_(map_eth_[map_eth_['ilk'] == prog])
    else:
        df_new = find_cr_rate_(map_eth_[map_eth_['ilk'] == prog])
        cr_rate = pd.concat([cr_rate, df_new], ignore_index=True, sort=False)

100%|██████████| 1094/1094 [00:30<00:00, 35.43it/s]


In [116]:
cr_rate.to_csv('cr_rate.csv')

### Find a liquidate rate

In [ ]:
# To get liqidate ratio need to query from biqquery by SPOT ABI

'''
SELECT  block_timestamp, input
FROM    bigquery-public-data.crypto_ethereum.traces
WHERE   DATE(block_timestamp) >= "2019-11-12"
        and DATE(block_timestamp) < "2023-08-01"
        and trace_address IS NOT NULL
        and call_type = "call"
        and to_address = LOWER("0x65c79fcb50ca1594b025960e539ed7a9a6d434a3")
        and status = 1
'''

In [118]:
liq_rate = pd.read_csv('liquidate_rate_jul2023.csv')
liq_rate

,block_timestamp,input
0,2021-12-09 14:00:24 UTC,0x1a0b287e47554e49563344414955534443322d410000...
1,2021-10-31 14:58:29 UTC,0x1a0b287e5a52582d4100000000000000000000000000...
2,2021-10-31 14:57:53 UTC,0x1a0b287e4241542d4100000000000000000000000000...
3,2021-10-31 12:00:43 UTC,0xd9638d364554482d4300000000000000000000000000...
4,2021-10-31 12:00:34 UTC,0x495d32cb
...,...,...
562803,2021-10-15 07:01:10 UTC,0x1504460f5a52582d4100000000000000000000000000...
562804,2021-10-15 17:01:09 UTC,0x1504460f5a52582d4100000000000000000000000000...
562805,2021-10-13 07:01:03 UTC,0x1504460f5a52582d4100000000000000000000000000...
562806,2021-10-13 08:00:25 UTC,0x1504460f5a52582d4100000000000000000000000000...


In [119]:
abi_vat = open("SPOT.txt", 'r').read()
w3 = Web3()
contract = w3.eth.contract(abi=abi_vat)

In [120]:
def decode_spot(tx_input):

    index = ['fn_name', 'args', 'timestamp']
    result = contract.decode_function_input(tx_input['input'])

    fn_name = result[0].fn_name
    fn_args = result[1]

    block_timestamp = tx_input['block_timestamp']

    return pd.Series(data=(fn_name, fn_args, block_timestamp), index=index)

In [121]:
tqdm.pandas()
res_liq_rate = liq_rate.progress_apply(decode_spot, axis=1)

100%|██████████| 562808/562808 [17:14<00:00, 543.99it/s] 


In [122]:
res_liq_rate.to_csv('decoded_liq_rate.csv')

In [123]:
file_result = res_liq_rate[res_liq_rate['fn_name']=='file']
file_result = file_result.reset_index(drop=True)

In [141]:
def get_arg_spot(df):
  cols = ['timestamp', 'ilk', 'what', 'rate']
  time = df['timestamp']
  dic = df['args']

  ilk = str(dic.get('ilk', None))
  ilk = ilk.replace("b'", "'")
  ilk = ilk.replace("\\x00", "")
  ilk = ilk.replace("\'", "\"")

  what = str(dic.get('what', None))
  what = what.replace("b'", "'")
  what = what.replace("\\x00", "")
  what = what.replace("\'", "\"")

  rate = dic.get('data', None)

  return pd.Series(data=(time, ilk, what, rate), index=cols)

In [142]:
tab_liq = file_result.progress_apply(get_arg_spot, axis=1)

 81%|████████  | 564/697 [00:00<00:00, 2907.45it/s]

100%|██████████| 697/697 [00:00<00:00, 2315.30it/s]


In [144]:
tab_liq_sort = tab_liq.sort_values(by='timestamp')
cond1 = tab_liq_sort['what'] == '"mat"'
cond2 = (tab_liq_sort['ilk'] == '"ETH-A"') | (tab_liq_sort['ilk'] == '"GUNIV3DAIUSDC2-A"') | (tab_liq_sort['ilk'] == '"WBTC-A"') | (tab_liq_sort['ilk'] == '"ETH-C"') | (tab_liq_sort['ilk'] == '"ETH-B"')
tab_liq_sort_cond = tab_liq_sort[cond1 & cond2]
tab_liq_sort_cond['rate'] = tab_liq_sort_cond['rate'].apply(lambda x: x/1e27)

In [13]:
data_tab = pd.read_csv('cr_rate.csv')
data_tab.drop(columns=['Unnamed: 0'], inplace=True)
data_tab['timestamp'] = pd.to_datetime(data_tab['timestamp'])
data_tab['Liquidate_Rate'] = np.nan

In [14]:
data_tab_eth_a = data_tab[data_tab['ilk'] == 'ETH-A']
data_tab_wbtc_a = data_tab[data_tab['ilk'] == 'WBTC-A']
data_tab_eth_b = data_tab[data_tab['ilk'] == 'ETH-B']
data_tab_eth_c = data_tab[data_tab['ilk'] == 'ETH-C']
data_tab_gun_a = data_tab[data_tab['ilk'] == 'GUNIV3DAIUSDC2-A']

In [15]:
change_rate = Timestamp('2021-08-27 17:02:30+0000', tz='UTC')
condition = data_tab_eth_a['timestamp'] < change_rate

data_tab_eth_a.loc[condition, 'Liquidate_Rate'] = 1.5
data_tab_eth_a.loc[~condition, 'Liquidate_Rate'] = 1.45

In [16]:
change_rate = Timestamp('2021-08-27 17:02:30 UTC', tz='UTC')
condition = data_tab_wbtc_a['timestamp'] < change_rate

data_tab_wbtc_a.loc[condition, 'Liquidate_Rate'] = 1.5
data_tab_wbtc_a.loc[~condition, 'Liquidate_Rate'] = 1.45

In [17]:
change_rate = Timestamp('2021-08-27 17:02:30 UTC', tz='UTC')
condition = data_tab_eth_c['timestamp'] < change_rate

data_tab_eth_c.loc[condition, 'Liquidate_Rate'] = 1.75
data_tab_eth_c.loc[~condition, 'Liquidate_Rate'] = 1.70

In [18]:
change_rate = Timestamp('2022-01-24 20:27:55 UTC', tz='UTC')
condition = data_tab_gun_a['timestamp'] < change_rate

data_tab_gun_a.loc[condition, 'Liquidate_Rate'] = 1.05
data_tab_gun_a.loc[~condition, 'Liquidate_Rate'] = 1.02

In [19]:
data_tab_eth_b['Liquidate_Rate'] = 1.3

In [20]:
data_all_ilk_all_usr = pd.concat([data_tab_eth_a, data_tab_wbtc_a, data_tab_eth_b, data_tab_eth_c, data_tab_gun_a], ignore_index=True, sort=False)

In [21]:
data_all_ilk_all_usr.to_csv('data_all_ilk_all_usr.csv')

In [7]:
data_all_ilk_all_usr = pd.read_csv('data_all_ilk_all_usr.csv')

## The datasets for one user per each ilk

In [210]:
data_tab_eth_a = data_tab_eth_a[data_tab_eth_a['usr'] == '0x931dBd7001D14112D17304B78d305c4FE317E571']
data_tab_wbtc_a = data_tab_wbtc_a[data_tab_wbtc_a['usr'] == '0x106DDC9252D79a0a3D1eD119e599F8ce4aA454eA']
data_tab_eth_b = data_tab_eth_b[data_tab_eth_b['usr'] == '0x6ab5641ad280A0881D649e0A12878E35Ca21F8DD']
data_tab_eth_c = data_tab_eth_c[data_tab_eth_c['usr'] == '0xF376b95Ed0AB7AC14eeA23648537Ce3A68b50648']
data_tab_gun_a = data_tab_gun_a[data_tab_gun_a['usr'] == '0x06Cd2a1595c7A2251D9963c813ed50Dd000508b8']

In [213]:
data_tab_eth_a.to_csv('data_tab_eth_a.csv')
data_tab_wbtc_a.to_csv('data_tab_wbtc_a.csv')
data_tab_eth_b.to_csv('data_tab_eth_b.csv')
data_tab_eth_c.to_csv('data_tab_eth_c.csv')
data_tab_gun_a.to_csv('data_tab_gun_a.csv')

In [215]:
data_all_ilk = pd.concat([data_tab_eth_a, data_tab_wbtc_a, data_tab_eth_b, data_tab_eth_c, data_tab_gun_a], ignore_index=True, sort=False)

In [217]:
data_all_ilk.to_csv('complete_eth_tab.csv')

In [212]:
table_eth_a = data_tab_eth_a.reset_index(drop=True)
table_eth_a['Liquidate_Rate'] = table_eth_a['Liquidate_Rate'].apply(lambda x: str(x))

table_wbtc_a = data_tab_wbtc_a.reset_index(drop=True)
table_wbtc_a['Liquidate_Rate'] = table_wbtc_a['Liquidate_Rate'].apply(lambda x: str(x))

table_eth_b = data_tab_eth_b.reset_index(drop=True)
table_eth_b['Liquidate_Rate'] = table_eth_b['Liquidate_Rate'].apply(lambda x: str(x))

table_eth_c = data_tab_eth_c.reset_index(drop=True)
table_eth_c['Liquidate_Rate'] = table_eth_c['Liquidate_Rate'].apply(lambda x: str(x))

table_gun_a = data_tab_gun_a.reset_index(drop=True)
table_gun_a['Liquidate_Rate'] = table_gun_a['Liquidate_Rate'].apply(lambda x: str(x))
